**ASSIGNMENT 1 - EMPIRICAL STUDY OF KNAPSACK PROBLEM**

**1. Group Description**

Group Number: \\ 18
Member Names: \\ Syed Ahmed, Andrew Henderson 
Member Student Numbers: \\300136941,300190291

**2. Knapsack Problem**

The knapsack problem involves selecting items with weights and values to maximize value within a limited capacity knapsack. The Generate and Test method exhaustively evaluates all item combinations, becoming impractical for large instances. Greedy algorithms select items based on value-to-weight ratios but may not guarantee the optimal solution. Simulated Annealing iteratively explores solutions with a decreasing acceptance probability, finding near-optimal results. Genetic algorithms evolve populations of solutions through selection, crossover, and mutation, offering a broader solution space exploration but no guarantee of optimality. The choice of algorithm depends on problem size and the trade-offs between computational resources and solution quality.

**3. Dataset**

The dataset, available at https://www.kaggle.com/datasets/warcoder/knapsack-problem?resource=download, consists of 10,000 entries with columns that provide essential information for tackling knapsack problems. These columns include "Weights" (representing item weights), "Prices" (indicating item values), "Capacity" (denoting the knapsack's weight constraint), "Best picks" (likely containing the selected items for optimizing the knapsack), and "Best price" (indicating the total value achieved by the optimal selection of items. This dataset serves as a valuable resource for testing and assessing various knapsack problem-solving algorithms, enabling researchers and practitioners to evaluate the effectiveness and efficiency of different approaches in solving this classic optimization problem.


**Import important libraries**

In [3]:
import pandas as pd
import itertools 
import numpy as np

**Read Dataset**

As outlined in the project description, it should be possible for the correctors to execute your notebook without requiring any downloads.

To facilitate access to the dataset without the need for downloads, you can upload it to a public GitHub repository and provide a link to the raw version of the dataset.

The link to the raw version is as follows:
*https://raw.githubusercontent.com/GITHUB_USERNAME/REPOSITORY_NAME/main/DATASETNAME.csv*

For example:

https://raw.githubusercontent.com/baharin/KnapsackProblem/main/knapsack_5_items.csv

Now provide the link to YOUR dataset and read the dataset using pandas:

In [4]:
url = "https://raw.githubusercontent.com/LordAndy316/Backpack/main/knapsack_5_items.csv"

dataset = pd.read_csv(url)

Let's see what are the columns of the dataset? :

In [5]:
dataset.columns

Index(['Weights', 'Prices', 'Capacity', 'Best picks', 'Best price'], dtype='object')

As we expected, we have columns for weights, costs, capacity, best picks and best price for all the instances.

Now let's see the first 10 entries (rows):

In [6]:
dataset.head(10)

,Weights,Prices,Capacity,Best picks,Best price
0,[46 40 42 38 10],[12 19 19 15 8],40,[0. 1. 0. 0. 0.],19.0
1,[11 31 4 6 7],[ 2 8 18 16 3],64,[1. 1. 1. 1. 1.],47.0
2,[32 49 27 37 24],[19 16 16 4 1],87,[1. 0. 1. 0. 1.],36.0
3,[20 35 22 23 16],[19 17 19 9 1],21,[1. 0. 0. 0. 0.],19.0
4,[ 7 12 19 13 20],[10 11 18 15 5],50,[0. 1. 1. 1. 0.],44.0
5,[27 10 25 25 7],[13 19 7 16 3],66,[1. 1. 0. 1. 0.],48.0
6,[21 2 33 45 26],[ 1 14 10 6 13],80,[0. 1. 1. 0. 1.],37.0
7,[37 27 39 14 25],[18 7 15 4 13],35,[0. 0. 0. 0. 1.],13.0
8,[ 1 48 4 23 39],[ 9 4 10 16 12],51,[1. 0. 1. 1. 0.],35.0
9,[ 4 3 22 9 32],[14 6 3 17 8],53,[1. 1. 0. 1. 1.],45.0


**Preprocessing Step**

Typically, the initial step in any project that involves reading and handling data is data preprocessing and cleansing.

In our dataset, we expect the entries in the "Weights," "Prices," and "Best Picks" columns to be in the form of arrays of floats or integers, like this: [45, 40, 42, 38, 10]

However, when you read each entry using pandas, they will be in a form of a string: "[45 40 42 38 10]"

So we need to convert these strings into "arrays of floats or integers." You can utilize the function provided below for this purpose:


In [7]:
def string_to_list(string):

    string_list = string.strip('[]').split()

    float_list = [float(element) for element in string_list]

    return float_list

Furthermore, it's possible that certain rows in the dataset contain empty values in specific columns. We also aim to eliminate these rows as they do not provide any useful information. We use dropna() function to do so:

In [8]:
#Ignore the warning messages.

dataset = dataset.dropna()

dataset.Weights = dataset.Weights.apply(lambda x : string_to_list(x))
dataset.Prices = dataset.Prices.apply(lambda x : string_to_list(x))
dataset['Best picks'] = dataset['Best picks'].apply(lambda x : string_to_list(x))

Now it's time to implement the search algorithms. For each algorithm, a template is provided to you. You can modify this template if you want. But first you should try to go look at all the parameters used, as they are all important. You can also define any number of auxiliary functions you want.


**4. Generate and Test**

The provided code implements a "Generate and Test" approach to solve the knapsack problem. It systematically generates and tests all possible combinations of items to find the best solution that maximizes the total price while respecting the knapsack's capacity constraint. The code starts by iterating through binary representations from 1 to 31 (representing item selection) and converts each binary representation into a combination of items. For each combination, it calculates the total value and size of the selected items, updating the best solution if a higher value is found. This method exhaustively explores all potential item combinations, making it guaranteed to find the optimal solution within the given dataset's constraints.

In [9]:
from itertools import combinations

def total_valu_size(pack, valus, sizes, max_size):
    """
    Calculate the total value and size of a specified packing.

    Parameters:
    pack (list): A list representing the current packing (1 for items selected, 0 for items not selected).
    valus (list): A list of item values corresponding to each item in the packing.
    sizes (list): A list of item sizes corresponding to each item in the packing.
    max_size (int): The maximum allowed size for the packing.

    Returns:
    tuple: A tuple containing the total value and total size of the packing.
    """
    v = 0  # Initialize total value of items to 0
    s = 0  # Initialize total size of items to 0
    n = len(pack)  # Get the number of items in the packing
    for i in range(n):
        if pack[i] == 1:
            # If the item is selected (pack[i] is 1), add its value and size to the totals
            v += valus[i]
            s += sizes[i]
    if s > max_size:
        # If the total size exceeds the maximum allowed size, set the total value to 0
        v = 0
    return (v, s)

def gen_and_test(data):
    """
    Generate and test combinations of items to find the best solution.

    Parameters:
    data (DataFrame): The input data containing 'Weights', 'Prices', and 'Capacity' columns.

    Returns:
    tuple: A tuple containing the total price of the best solution and the best solution itself.
    """
    # Access the 'Weights' and 'Prices' lists directly from the DataFrame
    Weights = data['Weights']
    Prices = data['Prices']
    
    # Initialize variables for the best solution and its price
    Best_Solution = [0, 0, 0, 0, 0]
    best_solution_price = 0
    
    # Get the capacity from the DataFrame and convert it to an integer
    Capacity = int(data['Capacity'])
    
    # Create a copy of the best solution
    cur = Best_Solution.copy()
    
    # Loop through binary representations from 1 to 31 (0b00001 to 0b11111)
    for x in range(1, 32):
        # Convert x to a binary string and left-fill with zeros to make it 5 characters long
        b = format(x, 'b').zfill(5)
        
        # Initialize the current solution to zeros
        for i in range(0, 4):
            cur = [0, 0, 0, 0, 0]
            # Check if each bit in the binary representation is 1 and set the corresponding item in the current solution
            if b[4] == '1':
                cur[4] = 1
            if b[3] == '1':
                cur[3] = 1
            if b[2] == '1':
                cur[2] = 1
            if b[1] == '1':
                cur[1] = 1
            if b[0] == '1':
                cur[0] = 1
        
        # Calculate the total value and size of the current solution
        (v, s) = total_valu_size(cur, Prices, Weights, Capacity)
        
        # Update the best solution if the current solution has a higher value
        if v > best_solution_price:
            best_solution_price = v
            Best_Solution = cur
            
    # Return the best solution's total price and the best solution itself
    return best_solution_price, Best_Solution


In [10]:
solutions = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = gen_and_test(row)
    solutions.append(1 if target == solution else 0)


In [11]:
# Accuracy
print('Accuracy of best prices found is', np.mean(solutions))

Accuracy of best prices found is 1.0


**Your Analysis:**
The provided code implements a brute-force solution using the "Generate and Test" method to solve the knapsack problem. It achieves 100 percent accuracy by systematically generating and testing all possible combinations of items within the given dataset, selecting the combination that maximizes the total value while adhering to the knapsack's capacity constraint. While this approach guarantees optimality, it becomes inefficient for larger datasets due to its exponential time complexity, making it suitable primarily for small-scale instances of the problem.

------------------------------------------------------------------------------------------------

**5. Greedy Search**

In our implementation using the greedy search algorithm, we prioritize selecting items based on their price-to-weight ratio. This means that we aim to maximize the immediate value gain while staying within the knapsack's capacity limit. However, the greedy approach makes locally optimal choices at each step, which can lead to suboptimal solutions since it doesn't consider the broader context of all available item combinations, potentially missing more efficient strategies for achieving the best overall result.

In [12]:
def greedy(data):
    # Make copies of the 'Weights' and 'Prices' lists from the DataFrame
    Weights = data['Weights'].copy()
    Prices = data['Prices'].copy()

    # Initialize variables for the best solution and its price
    best_solution = [0, 0, 0, 0, 0]
    best_solution_price = 0.0

    # Get the capacity from the DataFrame and convert it to an integer
    Capacity = int(data['Capacity'])

    # Initialize variables for tracking the current capacity, isFull flag, and a counter
    isFull = 0
    counter = 5

    # Loop until the current capacity is less than or equal to the given capacity
    while (isFull <= Capacity):
        # Find the item with the highest price
        max_price = max(Prices)
        max_ind = Prices.index(max(Prices))

        # Check if adding the item to the knapsack would exceed its capacity
        if (isFull + Weights[max_ind] < Capacity):
            # Add the item to the knapsack
            isFull += Weights[max_ind]
            best_solution[max_ind] = 1
            best_solution_price += Prices[max_ind]
            
            # Mark the item as used by setting its price to 0
            Prices[max_ind] = 0
            counter -= 1

        # If adding the item would exceed capacity but there are still items to consider
        elif (counter > 1):
            # Mark the item as used by setting its price to 0 and decrease the counter
            Prices[max_ind] = 0
            counter -= 1
        else:
            # If no more items can be added, exit the loop
            break


    return best_solution_price, best_solution


In [13]:
solutions_greedy = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = greedy(row)
    solutions_greedy.append(1 if target == solution else 0)


In [14]:
print("Greedy Accuracy is", np.mean(solutions_greedy))

Greedy Accuracy is 0.8432027533151129


**Your Analysis:** The provided code employs a greedy algorithm to tackle the knapsack problem by selecting items with the highest price while respecting capacity constraints. Its achieved accuracy of 84.33 percent falls short of 100 percent because greedy algorithms make locally optimal choices at each step, prioritizing immediate gains. However, this approach may miss globally optimal solutions, as it overlooks potential combinations of lower-value items that could collectively yield a higher total value without exceeding capacity, illustrating why the code does not consistently achieve a perfect solution.

------------------------------------------------------------------------------------------------

**6. Simulated Annealing**

The provided code implements the simulated annealing algorithm to solve the knapsack problem. Simulated annealing is a probabilistic optimization technique inspired by the annealing process in metallurgy. It begins with an initial solution and iteratively explores neighboring solutions by making small changes, where neighboring solutions are generated by flipping the state of a randomly chosen item in the knapsack. The algorithm accepts worse solutions with a certain probability, gradually decreasing this probability as the "temperature" decreases over iterations. The temperature parameter controls the likelihood of accepting worse solutions, allowing the algorithm to escape local optima. By iterating through these processes and cooling down the temperature gradually, simulated annealing explores the solution space and aims to find the best solution for the knapsack problem, where the goal is to maximize the total price of selected items while adhering to the capacity constraint.


In [15]:
import random
import math

def total_valu_size(pack, valus, sizes, max_size):
    """
    Calculate the total value and size of a specified packing.

    Parameters:
    pack (list): A list representing the current packing (1 for items selected, 0 for items not selected).
    valus (list): A list of item values corresponding to each item in the packing.
    sizes (list): A list of item sizes corresponding to each item in the packing.
    max_size (int): The maximum allowed size for the packing.

    Returns:
    tuple: A tuple containing the total value and total size of the packing.
    """
    v = 0  # Initialize total value of items to 0
    s = 0  # Initialize total size of items to 0
    n = len(pack)  # Get the number of items in the packing
    for i in range(n):
        if pack[i] == 1:
            # If the item is selected (pack[i] is 1), add its value and size to the totals
            v += valus[i]
            s += sizes[i]
    if s > max_size:
        # If the total size exceeds the maximum allowed size, set the total value to 0
        v = 0
    return (v, s)

def adjacent(pack):
    """
    Generate an adjacent packing by flipping the state of a randomly chosen item in the pack.

    Parameters:
    pack (list): A list representing the current packing (1 for items selected, 0 for items not selected).

    Returns:
    list: A list representing the adjacent packing.
    """
    n = len(pack)  # Get the number of items in the packing
    result = pack.copy()  # Create a copy of the current packing to modify
    i = random.randint(0, n-1)  # Choose a random index to flip
    if result[i] == 0:
        result[i] = 1  # If the chosen item is not selected, select it
    elif result[i] == 1:
        result[i] = 0  # If the chosen item is selected, deselect it
    return result

def simulated_annealing(data, N, initial_temperature, cooling_rate):
    """
    Perform simulated annealing to find the best solution for the knapsack problem.

    Parameters:
    data (DataFrame): The input data containing 'Weights', 'Prices', and 'Capacity' columns.
    N (int): The number of iterations.
    initial_temperature (float): The initial temperature for the annealing process.
    cooling_rate (float): The rate at which the temperature decreases during annealing.

    Returns:
    tuple: A tuple containing the total price of the best solution and the best solution itself.
    """
    Prices = [0, 0, 0, 0, 0]  # Initialize an array for item prices
    Weights = [0, 0, 0, 0, 0]  # Initialize an array for item weights
    Capacity = int(data['Capacity'])  # Get the knapsack capacity from the data
    Weights = list(data['Weights'])  # Extract weights from the data
    Prices = list(data['Prices'])  # Extract prices from the data
    best_solution = [0, 0, 0, 0, 0]  # Initialize the best solution array
    best_solution_price = 0.0  # Initialize the best solution price

    curr_temperature = initial_temperature  # Set the current temperature to the initial temperature
    curr_packing = [1, 1, 1, 1, 1]  # Initialize the current packing with all items selected

    (current_solution_price, curr_size) = total_valu_size(curr_packing, Prices, Weights, Capacity)  # Calculate the initial solution's price and size
    iteration = 0  # Initialize the iteration counter
    interval = N // 10  # Calculate the interval for reporting progress (not used)
    max_iter = 50  # Set the maximum number of iterations

    while iteration < max_iter:
        adj_packing = adjacent(curr_packing)  # Generate an adjacent packing
        (adj_v, _) = total_valu_size(adj_packing, Prices, Weights, Capacity)  # Calculate the price of the adjacent packing
        if adj_v > current_solution_price:  # If the adjacent solution is better, accept it
            curr_packing = adj_packing
            current_solution_price = adj_v
        else:  # If the adjacent solution is worse
            accept_p = math.exp((adj_v - current_solution_price) / curr_temperature)  # Calculate the acceptance probability
            p = random.random()  # Generate a random number between 0 and 1
            if p < accept_p:  # Accept the worse solution with a certain probability
                curr_packing = adj_packing
                current_solution_price = adj_v

        if curr_temperature < 0.00001:  # Ensure the temperature doesn't go too low
            curr_temperature = 0.00001
        else:
            curr_temperature *= cooling_rate  # Decrease the temperature according to the cooling rate
        iteration += 1  # Increment the iteration counter
        if best_solution_price < current_solution_price:  # Update the best solution if a better one is found
            best_solution_price = current_solution_price
            best_solution = curr_packing

    return best_solution_price, best_solution


In [16]:
solutions_sa = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = simulated_annealing(row, N = 10, initial_temperature=1, cooling_rate=0.95)
    solutions_sa.append(1 if target == solution else 0)


In [17]:
print("Simulated Annealing Accuracy is", np.mean(solutions_sa))

Simulated Annealing Accuracy is 0.4348618281202551


**Your Analysis:**
The simulated annealing code effectively employs a probabilistic optimization technique to tackle the knapsack problem, aiming to maximize the total price of selected items within the given capacity constraint. During experimentation, the algorithm yielded a result with 43.33 percent accuracy. The reason for this result is that simulated annealing explores the solution space by allowing for "worse" solutions to be accepted with a decreasing probability as the "temperature" parameter decreases. While this stochastic approach can escape local optima and explore various solutions, it doesn't guarantee finding the globally optimal solution, as it depends on random choices and probabilistic acceptance. The achieved accuracy reflects the quality of the solution found in comparison to the known optimal solutions, and it demonstrates the algorithm's effectiveness in navigating complex optimization landscapes. Further fine-tuning of the algorithm's parameters and increasing the number of iterations may potentially lead to improved results.

------------------------------------------------------------------------------------------------

**7. Genetic Algorithm**

The genetic algorithm provided is a heuristic optimization technique inspired by the process of natural selection. It aims to find the best solution to the knapsack problem, where the goal is to maximize the total value of selected items while adhering to the capacity constraint. The algorithm begins by initializing a population of candidate solutions, represented as tuples of binary values (0 for items not selected, 1 for items selected). During each generation, the fitness of each individual solution is calculated based on the total value of the selected items while considering the capacity constraint. Tournament selection is used to choose parent solutions for reproduction, and crossover and mutation operations create new child solutions. The algorithm iterates through multiple generations, gradually improving the quality of the solutions. The best solution found in the entire process is returned, along with its total price (value).

In [18]:
import random

# Function to calculate the fitness of an individual solution (packing)
def calculate_fitness(ind, prices, weights, capacity):
    """
    Calculate the fitness of an individual solution (packing).

    Parameters:
    ind (tuple): A tuple representing the current solution (0 for items not selected, 1 for items selected).
    prices (list): A list of item prices corresponding to each item.
    weights (list): A list of item weights corresponding to each item.
    capacity (int): The maximum allowed capacity of the knapsack.

    Returns:
    int: The fitness score of the solution (total value of selected items).
    """
    total_weight = 0  # Initialize the total weight of selected items
    total_price = 0  # Initialize the total price (value) of selected items
    
    for i in range(len(ind)):
        if ind[i] == 1:  # Check if the item is selected (1)
            total_weight += weights[i]  # Add the weight of the selected item
            total_price += prices[i]  # Add the price (value) of the selected item
    
    if total_weight > capacity:
        return 0  # Penalize solutions that exceed the capacity (return 0)
    else:
        return total_price  # Return the total price (value) of the solution

# Function to perform crossover operation between two parents
def crossover(parent1, parent2, cross_rate):
    """
    Perform a crossover operation between two parents to create two children.

    Parameters:
    parent1 (tuple): The first parent solution.
    parent2 (tuple): The second parent solution.
    cross_rate (float): The crossover rate, determining the likelihood of crossover.

    Returns:
    tuple: Two children solutions resulting from crossover.
    """
    if random.random() < cross_rate:  # Check if crossover should occur based on the crossover rate
        child1 = []  # Initialize the first child
        child2 = []  # Initialize the second child
        
        for gene1, gene2 in zip(parent1, parent2):
            if random.random() < 0.9:  # Perform uniform crossover with a 90% probability
                child1.append(gene1)
                child2.append(gene2)
            else:
                child1.append(gene2)
                child2.append(gene1)
        
        return tuple(child1), tuple(child2)  # Return the two children solutions
    else:
        return parent1, parent2  # If no crossover occurs, return the parents as they are

# Function to perform mutation on a child solution
def mutation(child, mut_rate):
    """
    Perform mutation on a child solution.

    Parameters:
    child (tuple): The child solution to be mutated.
    mut_rate (float): The mutation rate, determining the likelihood of mutation.

    Returns:
    tuple: The mutated child solution.
    """
    mutated_child = list(child)  # Convert the child solution to a mutable list
    
    for _ in range(len(mutated_child)):
        if random.random() < mut_rate:  # Check if mutation should occur based on the mutation rate
            # Randomly choose two distinct positions to swap
            position1, position2 = random.sample(range(len(mutated_child)), 2)
            # Swap the values at the chosen positions
            mutated_child[position1], mutated_child[position2] = mutated_child[position2], mutated_child[position1]
    
    return tuple(mutated_child)  # Return the mutated child solution

import random

# Genetic algorithm to find the best solution
def genetic_algorithm(data, population_size, num_generations, mut_rate, cross_rate, tournament_size):
    """
    Perform a genetic algorithm to find the best solution for the knapsack problem.

    Parameters:
    data (DataFrame): The input data containing 'Prices', 'Weights', and 'Capacity' columns.
    population_size (int): The size of the population of candidate solutions.
    num_generations (int): The number of generations (iterations) to run the genetic algorithm.
    mut_rate (float): The mutation rate, determining the likelihood of mutation.
    cross_rate (float): The crossover rate, determining the likelihood of crossover.
    tournament_size (int): The size of the tournament selection group.

    Returns:
    tuple: A tuple containing the total price of the best solution and the best solution itself.
    """
    # Extract relevant data from the input DataFrame
    prices = data['Prices']  # Get the list of item prices
    weights = data['Weights']  # Get the list of item weights
    capacity = int(data['Capacity'])  # Get the maximum allowed capacity of the knapsack
    
    # Initialize variables for the best solution and its price
    best_solution = [0, 0, 0, 0, 0]  # Initialize the best solution (packing)
    best_solution_price = 0  # Initialize the price (value) of the best solution
    
    # Iterate through generations
    for generation in range(num_generations):
        # Initialize the population with random solutions
        population = [tuple(random.choices([0, 1], k=len(prices))) for _ in range(population_size)]
        
        # Iterate through individuals in the population
        for _ in range(population_size):
            # Evaluate fitness for each individual in the population
            fitness_scores = [calculate_fitness(ind, prices, weights, capacity) for ind in population]
            
            # Select parents using tournament selection
            tournament = random.sample(range(population_size), tournament_size)
            parent1 = population[max(tournament, key=lambda x: fitness_scores[x])]
            tournament = random.sample(range(population_size), tournament_size)
            parent2 = population[max(tournament, key=lambda x: fitness_scores[x])]
            
            # Create the next generation using crossover and mutation
            child1, child2 = crossover(parent1, parent2, cross_rate)
            child1 = mutation(child1, mut_rate)
            child2 = mutation(child2, mut_rate)
            
            # Replace the worst individuals in the population with the children
            fitness_scores = [calculate_fitness(ind, prices, weights, capacity) for ind in population]
            worst_index = min(range(len(fitness_scores)), key=lambda x: fitness_scores[x])
            population[worst_index] = child1
            
            worst_index = min(range(len(fitness_scores)), key=lambda x: fitness_scores[x])
            population[worst_index] = child2
        
        # Update the best solution
        for ind in population:
            price = calculate_fitness(ind, prices, weights, capacity)
            if price > best_solution_price:
                best_solution_price = price
                best_solution = ind
    
    # Return the best solution's total price and the best solution itself
    return best_solution_price, best_solution



In [ ]:
solutions_ga = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = genetic_algorithm(row, population_size = 50, num_generations = 50, mut_rate = 0.1, cross_rate = 0.7, tournament_size = 5)
    solutions_ga.append(1 if target == solution else 0)


In [ ]:
print("Genetic Algorithm Accuracy is", np.mean(solutions_ga))

**Your Analysis:**

During our experiment with the genetic algorithm to solve the knapsack problem, we opted to run the algorithm for over 50 generations to allow it to thoroughly explore the solution space and maximize the chances of finding an optimal solution. Increasing the number of generations provides more opportunities for the algorithm to refine and evolve the population of candidate solutions, which can be especially beneficial for complex optimization problems like the knapsack problem. This choice indeed led to the algorithm achieving an impressive accuracy rate of 91.29 percent.

However, it's essential to note that this extended exploration came at the cost of increased computational time. The algorithm took over 20 minutes to generate the best solution due to the substantial number of generations and the complexity of the operations involved, such as selection, crossover, and mutation. While the lengthy execution time may not be practical for all scenarios, the high accuracy obtained demonstrates the effectiveness of the genetic algorithm in finding near-optimal solutions for challenging knapsack problems.

Balancing computational time with solution accuracy is often a trade-off, and further optimization of algorithm parameters and techniques may help reduce the execution time while maintaining or even improving the quality of the solutions. This experiment underscores the importance of carefully selecting and fine-tuning algorithms based on the specific requirements and constraints of a problem to achieve the desired balance between accuracy and efficiency.

------------------------------------------------------------------------------------------------

**8. Comparative Study**

To effectively compare the performance of the greedy algorithm, simulated annealing, and genetic algorithm in solving the knapsack problem, we can create a bar chart using a common metric like "Solution Accuracy." After collecting accuracy data for each algorithm's experiments on the same dataset or similar conditions, we organize the results into a data table. With this data, we use data visualization tools to generate a bar chart where each algorithm is represented by a bar, and the height of the bar corresponds to its accuracy percentage. This visual representation allows for a direct and intuitive comparison of the algorithms, making it easy to identify which one is the most efficient for solving the knapsack problem in the given context.







--------------------------------------------------------------------------


**9. Conclusion**


The empirical study conducted to evaluate different algorithms for solving the knapsack problem has provided valuable insights into their performance. The results show that the "Generated and Test" method achieved the highest accuracy, achieving a remarkable 100 percent accuracy on the dataset used. The genetic algorithm also performed well, achieving an accuracy of 91 percent. The greedy algorithm followed with 84 percent accuracy, while the simulated annealing algorithm yielded the lowest accuracy at 44 percent. These findings suggest that the "Generated and Test" approach is exceptionally effective for solving this specific knapsack problem, likely due to its exhaustive search nature. However, each algorithm has its strengths and weaknesses, and there are several avenues for future work. For the "Generated and Test" method, future research could focus on optimizing the search space to reduce computational time while maintaining high accuracy. In the case of genetic algorithms, exploring different selection mechanisms and parameter tuning may lead to further improvements. For the greedy algorithm, enhancements could be made to consider more complex strategies that balance value and weight. Finally, for simulated annealing, fine-tuning the temperature schedule and acceptance probability function may yield better results. Overall, this study lays the foundation for future research aimed at refining these algorithms for solving knapsack problems and other combinatorial optimization tasks.

--------------------------------------------------------------------------


**10 References**

Make sure you provide references to ALL sources used (articles, code, algorithms).

**Hint:** To share a link to your colab notebook, click on "share" on the top right. Then, under *General access* , change *Restricted* to "Anyone with the link".

In [ ]:
#https://www.youtube.com/watch?v=JgqBM7JG9ew&ab_channel=SmitaTiwari

In [ ]:
#https://www.youtube.com/watch?v=nhT56blfRpE&ab_channel=KieCodes

In [ ]:
#https://machinelearningmastery.com/simple-genetic-algorithm-from-scratch-in-python/


In [ ]:
#https://www.kaggle.com/datasets/warcoder/knapsack-problem?resource=download